<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/gain_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def gain_analyzer(abf,spike_args =  {'spike_thresh':10, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2}, R2_thresh = 0.8, to_plot = 0):
    'Analyze Single ABF of increasing current injections for firing rate gain'
    'to_plot scales from 0:2, no plot, plot just the final fitting, plot every sweep for spike detection'

    results= {} # init results dict
    if len(abf.sweepList)<5: return results # not enough sweeps to analyze

    is_base, is_stim = protocol_baseline_and_stim(abf) # find base lines and stims
    stim_currents, spike_counts, spike_rates,_,v_before_stim, fire_dur = spikes_per_stim(abf,spike_args, mode='count', to_plot=to_plot)

    if sum(spike_counts)==0: return results   #if no spikes return none
    if_fit = fit_firing_gain( stim_currents, spike_counts, spike_rates ,to_plot=to_plot>0)


    results['Gain_(Hz/pA)']=if_fit['slope']
    results['Gain_R2']=if_fit['R2']
    results['Stim_Levels_(pA)']=stim_currents
    results['Spike_Counts']=spike_counts
    results['Firing_Duration_%']=fire_dur
    results['Gain_Vh']=v_before_stim

    return results

In [ ]:
'Functions for Basic analysis of firing rate gain parameters'
'requires: numpy (np), pandas (pd), pyabf (pyabf) , matplotlib'
# from matplotlib import pyplot as plt

def len_one(x):
    try:
        return len(x)
    except:
        return 1


def fit_firing_gain(stim_currents, spike_counts, spike_rates, to_plot=False):
    'Gathers the firing rate of each stimuli and fits the linear portion of the curve to return the Gain in Hz/pA (the slope)'
    is_pos_slope = np.diff(spike_counts,prepend=0)>0
    is_pos_slope = movmean(np.diff(spike_counts,prepend=0),4)>0
    peak_ind = np.where(spike_counts==np.max(spike_counts))[0]
    if len(peak_ind)>1:
        peak_ind = np.min(peak_ind)
    
    before_peak = np.arange(len(spike_counts))<peak_ind
    is_nonzero = spike_counts>0
    use_for_fit = np.logical_and.reduce((is_pos_slope,is_nonzero,before_peak))

    if_fit = {}
    if_fit['stim_currents'] = stim_currents
    if_fit['spike_rates'] = spike_rates
    if 0 == np.sum(spike_rates):
        # print('no spikes detected')
        if_fit['slope'] = np.nan
        if_fit['intercept'] = np.nan
        if_fit['R2'] = 0
        return if_fit

    if np.sum(spike_rates>0)<3:
        # print('not enough spikes generated')
        if_fit['slope'] = np.nan
        if_fit['intercept'] = np.nan
        if_fit['R2'] = 0
        return if_fit

    if_fit['slope'], if_fit['intercept'] , r_value, p_value, std_err = stats.linregress(stim_currents[use_for_fit], spike_rates[use_for_fit])
    if_fit['R2'] = r_value**2

    if to_plot:
        fig, ax = plt.subplots(1, figsize=[3,3])
        ax.scatter( if_fit['stim_currents'] ,if_fit['spike_rates'] )
        ax.plot( if_fit['stim_currents'], if_fit['slope']* if_fit['stim_currents']+if_fit['intercept'])
        ax.scatter( if_fit['stim_currents'][use_for_fit] ,if_fit['spike_rates'][use_for_fit], color='r' )
        ax.scatter(if_fit['stim_currents'][peak_ind],if_fit['spike_rates'][peak_ind], color='m')
        ax.set_xlabel('current')
        ax.set_ylabel('Spike Rate (Hz)')
        (min,max) = ax.get_ylim()
        ax.text(0, max/2, 'R**2='+str(round(if_fit['R2'],2)),fontsize='large')
        plt.show()
    return if_fit

def check_inactivation( time, trace, is_stim, sample_rate, dVds, inds, mean_spike_rate, to_plot=0 ):
    time_ms = time*1000
    sum_isi = None
    rel_firing_duration = None
    if len(inds)>0:
        stim_time = time_ms[np.where(is_stim)[0][0]]
        firing_duration = time[inds[-1]]
        rel_firing_duration = firing_duration /(np.max(time[is_stim]*1000)-stim_time)
    return rel_firing_duration
